In [20]:
#basic RNN
import tensorflow as tf

n_inputs =3
n_neurons = 5

X0 =  tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0, Wx) +b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

init = tf.global_variables_initializer()

In [21]:
import numpy as np

X0_batch = np.array([[0,1,2], [3,4,5], [6,7,8], [9,0,1]])
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0:X0_batch, X1:X1_batch})
    

In [22]:
print(Y0_val)

[[ 0.736362    0.56424904 -0.9928428   0.99697137  0.9957681 ]
 [ 0.9999985   0.98514026 -1.          1.          1.        ]
 [ 1.          0.9995978  -1.          1.          1.        ]
 [ 0.99999887  0.9990503  -0.99999994  0.99999994  0.99580586]]


In [23]:
print(Y1_val)

[[ 1.          0.9999874  -1.          1.          1.        ]
 [ 0.69753206  0.6118082  -0.90616906  0.9795825  -0.8114883 ]
 [ 1.          0.9986853  -1.          1.          1.        ]
 [ 0.9999879   0.9524181  -0.99999994  0.99996644  0.9973081 ]]


## BasicRNNCell

In [24]:
tf.reset_default_graph()
# static unrolling through time

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])
# we create input palceholders

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
#  we create BasicRNNCell which you can think of as a factory that creates copies of the cell to build the unrolled RNN

output_seqs,states = tf.contrib.rnn.static_rnn(
                     basic_cell, [X0, X1], dtype=tf.float32)
# we call static rnn which calls the BasicRNNCell creating two copies of cell


Y0, Y1 = output_seqs

In [25]:
# if there wer 50 time steps, it wouldnot be ery convenient to have ot 
# define 50 input placeholders and 50 output tensors

print(Y0)
print(Y1)



Tensor("rnn/basic_rnn_cell/Tanh:0", shape=(?, 5), dtype=float32)
Tensor("rnn/basic_rnn_cell/Tanh_1:0", shape=(?, 5), dtype=float32)


In [26]:
# using the packing sequences
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0:X0_batch, X1:X1_batch})

In [27]:
Y0_val

array([[-0.6220045 ,  0.3434742 ,  0.5967921 ,  0.5467581 , -0.11334981],
       [-0.8740386 ,  0.9972768 ,  0.987609  ,  0.9661833 , -0.81035984],
       [-0.96196574,  0.9999924 ,  0.9996922 ,  0.99798304, -0.972817  ],
       [-0.27229077,  0.9997892 ,  0.99158883,  0.99976593, -0.99935156]],
      dtype=float32)

In [28]:
Y1_val

array([[-0.62965757,  0.99999994,  0.9999669 ,  0.9991317 , -0.9964192 ],
       [ 0.6258266 ,  0.7319028 ,  0.891579  , -0.2367793 , -0.00984494],
       [ 0.22992855,  0.99999607,  0.9997939 ,  0.9622914 , -0.95374256],
       [ 0.55169165,  0.9969307 ,  0.97981477,  0.08551132, -0.7642403 ]],
      dtype=float32)

In [30]:
# from tensorflow_graph_in_jupyter import show_graph

# show_graph(tf.get_default_graph())

## Packing Sequences

In [31]:
# the following cide builds the same RNN again but thistime it takes asingle input 
# placeholder of hsape [None, n_steps, n_inputs ] where the first  fimensions is the mini batch size\
# Then it extract thelist of input sequences for each time step
# X_seqs is a Python list of n_steps tensors of shape [None,n_inputs]
# where once again the first dimension is the mini batch size
# To do this, we first swap using transpose function so that the time steps are now the first dimension 
# then we use the unstack function. The next twolines are the sam
# finally we merge the the output tensors into a single tensor using stack() function
# and we sqap the first two fimension to get a final outputs tensor
# of shape [None, n_steps, n_neurons]

n_steps =2
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps,n_inputs])
X_seqs = tf.unstack(tf.transpose(X,perm=[1,0,2]))
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs, dtype=tf.float32)
outputs = tf.transpose(tf.stack(output_seqs), perm=[1,0,2])

In [32]:
init = tf.global_variables_initializer()

In [34]:
X_batch= np.array([
    [[0,1,2], [9,8,7]],
    [[3,4,5], [0,0,0]],
    [[6,7,8], [6,5,4]],
    [[9,0,1], [3,2,1]]
])
g = tf.Graph()
with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X:X_batch})

# this stillgets one graph containing unit cell per time step
# If there were 50 time steps the graph would look pretty ugly
print(outputs_val)

# with large graph we will also get out of memory errors so that it can
# use them to compute gradients during the reverse pass
# thats why we using dynamic rnn

[[[ 0.80800104 -0.25818652  0.88132834  0.79292494 -0.08273872]
  [-0.5822327  -0.9999026   0.999995   -0.657447   -0.96924794]]

 [[ 0.8161503  -0.9309901   0.99883807  0.84217244 -0.7897056 ]
  [ 0.5496408  -0.85100687  0.21461645 -0.20366207  0.8525351 ]]

 [[ 0.82398736 -0.9956768   0.9999893   0.88048846 -0.9679276 ]
  [-0.33215857 -0.999035    0.9979041  -0.7774459  -0.4956271 ]]

 [[-0.9999593  -0.999779    0.14563009 -0.9997508  -0.93437034]
  [-0.87933636 -0.6990654  -0.65328014 -0.6297835  -0.6142405 ]]]


In [35]:
print(np.transpose(outputs_val, axes=[1,0,2])[1])

[[-0.5822327  -0.9999026   0.999995   -0.657447   -0.96924794]
 [ 0.5496408  -0.85100687  0.21461645 -0.20366207  0.8525351 ]
 [-0.33215857 -0.999035    0.9979041  -0.7774459  -0.4956271 ]
 [-0.87933636 -0.6990654  -0.65328014 -0.6297835  -0.6142405 ]]


In [2]:
import tensorflow as tf
tf.reset_default_graph()

n_steps =2
n_inputs = 3
n_neurons = 5

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

W0616 10:20:31.411031 12108 deprecation.py:323] From <ipython-input-2-6755c56d7f85>:10: BasicRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
W0616 10:20:31.414023 12108 deprecation.py:323] From <ipython-input-2-6755c56d7f85>:11: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0616 10:20:31.476860 12108 deprecation.py:323] From C:\installs\Anaconda\envs\tf_gpu\lib\site-packages\tensorflow_core\python\ops\rnn_cell_impl.py:456: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.
W0616 10:20:31.487831 121

In [3]:
init = tf.global_variables_initializer()

In [6]:
import numpy as np
X_batch =  np.array([
    [[0,1,2], [9,8,7]],
    [[3,4,5], [0,0,0]],
    [[6,7,8], [6,5,4]],
    [[9,0,1], [3,2,1]]
])

with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X:X_batch})

In [8]:
print(outputs_val)

# show_graph(tf.get_default_graph())

[[[-0.3075267   0.5299915   0.16965647  0.87184805  0.58252114]
  [-0.9981262   0.9964085  -0.09262892 -0.39578518  0.97315013]]

 [[-0.9239652   0.97164756  0.09598505  0.93754584  0.9429492 ]
  [ 0.47932312 -0.8938842   0.2604586  -0.45113435 -0.44469807]]

 [[-0.9941193   0.9986546   0.02124681  0.97010124  0.9934848 ]
  [-0.93894494  0.74343777 -0.0715929  -0.8035607   0.7878499 ]]

 [[-0.99964064  0.9620414  -0.99611646 -0.9999498  -0.9915379 ]
  [-0.41506955  0.73790824 -0.7738236  -0.97888213  0.9177495 ]]]


## Setting the sequence lengths

In [9]:
n_steps = 2
n_inputs = 3
n_neurons = 5

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)


In [10]:
seq_length = tf.placeholder(tf.int32, [None])
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32,
                                    sequence_length=seq_length)

W0616 10:26:55.937911 12108 deprecation.py:323] From C:\installs\Anaconda\envs\tf_gpu\lib\site-packages\tensorflow_core\python\ops\rnn.py:244: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
init = tf.global_variables_initializer()

In [13]:
X_batch = np.array([
    #step 0   step 1
    [[0,1,2], [9,8,7]], # instance 1
    [[3,4,5], [0,0,0]], # Padded Instance 2
    [[6,7,8], [6,5,4]], # instance 3
    [[9,0,1], [3,2,1]],  # instance 4
])

seq_length_batch = np.array([2,1,2,2])

In [15]:
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run(
        [outputs, states],feed_dict={X:X_batch, seq_length: seq_length_batch })
    

In [16]:
print(outputs_val)

[[[-0.11812522 -0.20228527  0.11452613  0.11399005  0.3338867 ]
  [-0.99728656 -0.9996351   0.75922894 -0.99910766  0.24618293]]

 [[-0.87457997 -0.94072527  0.44783533 -0.8127231   0.474531  ]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.9887129  -0.997192    0.69051933 -0.98316467  0.5945068 ]
  [-0.96373564 -0.9832916   0.7888061  -0.96899116 -0.5954966 ]]

 [[-0.9998727  -0.9999861  -0.9860169  -0.9999886  -0.65876055]
  [-0.8518146  -0.9395353   0.88231933 -0.6496556  -0.8854977 ]]]


In [17]:
print(states_val)

[[-0.99728656 -0.9996351   0.75922894 -0.99910766  0.24618293]
 [-0.87457997 -0.94072527  0.44783533 -0.8127231   0.474531  ]
 [-0.96373564 -0.9832916   0.7888061  -0.96899116 -0.5954966 ]
 [-0.8518146  -0.9395353   0.88231933 -0.6496556  -0.8854977 ]]


## Training a sequence classifier

In [20]:
tf.reset_default_graph()

n_steps =28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

# these are the two lines that are defining a RNN
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X # from feed_dict
                                    ,dtype=tf.float32)

# dynamic and static rnn bothe functions calls the BasicRNNCell call function per input
# createing two copise with n_neurons no. of connection between the layers
# it returns twoobjects  python list containing the output tensors foreach step and final states of the network
#in basic cell the final state is equal to the last output

logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y  # from feed_dict
                                                          , logits=logits)

loss= tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [22]:
(X_train,y_train), (X_test,y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28 * 28)/ 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28 * 28)/255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]


In [27]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch
        # this is a generator
        # multiple returns to single call

In [24]:
X_test = X_test.reshape((-1, n_steps,n_inputs))

In [28]:
n_epochs = 100
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            X_batch = X_batch.reshape((-1, n_steps,n_inputs ))
            sess.run(training_op,feed_dict={X:X_batch,y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test})
        print(epoch,"Last batch accuracy: ", acc_batch, "Test accuracy:", acc_test)

0 Last batch accuracy:  0.9266667 Test accuracy: 0.9357
1 Last batch accuracy:  0.98 Test accuracy: 0.9441
2 Last batch accuracy:  0.97333336 Test accuracy: 0.9637
3 Last batch accuracy:  0.9533333 Test accuracy: 0.9549
4 Last batch accuracy:  0.97333336 Test accuracy: 0.9657
5 Last batch accuracy:  0.98 Test accuracy: 0.9636
6 Last batch accuracy:  0.94666666 Test accuracy: 0.9664
7 Last batch accuracy:  0.98 Test accuracy: 0.9691
8 Last batch accuracy:  0.97333336 Test accuracy: 0.9743
9 Last batch accuracy:  0.98 Test accuracy: 0.9733
10 Last batch accuracy:  0.96666664 Test accuracy: 0.965
11 Last batch accuracy:  0.97333336 Test accuracy: 0.9682
12 Last batch accuracy:  0.96666664 Test accuracy: 0.9716
13 Last batch accuracy:  0.98 Test accuracy: 0.9656
14 Last batch accuracy:  0.9866667 Test accuracy: 0.9715
15 Last batch accuracy:  0.9866667 Test accuracy: 0.9696
16 Last batch accuracy:  0.9866667 Test accuracy: 0.9743
17 Last batch accuracy:  0.98 Test accuracy: 0.9742
18 Last 

### Multi Layer RNN

In [30]:
# multi layer RNN

tf.reset_default_graph()

n_steps = 28
n_inputs = 28
n_outputs = 10

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None,n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

In [31]:
n_neurons = 100
n_layers = 2

layers = [tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)for layer in range(n_layers)]
# layers is not required here

multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell(layers)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

W0616 11:33:53.245198 12108 deprecation.py:323] From <ipython-input-31-766197b0afd1>:7: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [32]:
states_concat = tf.concat(axis=1, values=states)

logits = tf.layers.dense(states_concat,n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [33]:
n_epochs = 10
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            X_batch = X_batch.reshape((-1,n_steps,n_inputs ))
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_batch = accuracy.eval(feed_dict={X:X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test, y: y_test})
        print(epoch, "last batch accuracy: ", acc_batch, "test_accuracy: ", acc_test)

0 last batch accuracy:  0.93333334 test_accuracy:  0.9233
1 last batch accuracy:  0.97333336 test_accuracy:  0.9529
2 last batch accuracy:  0.94 test_accuracy:  0.9631
3 last batch accuracy:  0.98 test_accuracy:  0.9662
4 last batch accuracy:  0.97333336 test_accuracy:  0.9651
5 last batch accuracy:  0.9866667 test_accuracy:  0.9756
6 last batch accuracy:  0.96666664 test_accuracy:  0.9717
7 last batch accuracy:  0.97333336 test_accuracy:  0.9719
8 last batch accuracy:  0.9866667 test_accuracy:  0.9744
9 last batch accuracy:  0.98 test_accuracy:  0.9779
